In [21]:


from dotenv import load_dotenv
load_dotenv(dotenv_path=".env.local")
import os 
from  openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



In [18]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec("aws", "us-east-1"))

In [20]:
import json 
data = json.load(open("reviews.json"))
data['reviews']
len(data['reviews'])

20

In [23]:
processed_data = [] 
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review["review"], 
        model='text-embedding-3-small'
    )
    embeddings = response.data[0].embedding 
    processed_data.append({ 
        "values" : embeddings, 
        "id": review["professor"], 
        "metadata" : {
            "review" : review["review"],
            "subject" : review["subject"],
            "stars" : review["stars"] 
        }
        })

In [25]:
index = pc.Index('rag')
index.upsert( 
             vectors=processed_data, 
             namespace="ns1"
             )